## Evaluate the trained network (Week 12) - Step 5

####**Designed by Joon Son Chung, November 2020**

This is based on https://github.com/joonson/face_trainer.

In this step, we evaluate the model trained in Step 4. Change `initial_model` to the saved model that you want to evaluate. You should only test the model that gives the best validation performance during training.



In [ ]:
from google.colab import drive
from zipfile import ZipFile
from tqdm import tqdm
import os, glob, sys, shutil, time
import torch
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image

# mount Google Drive
drive.mount('/content/drive', force_remount=True)

# path of the data directory relative to the home folder of Google Drive
GDRIVE_HOME = '/content/drive/My Drive'
FOLDER      = 'MLVU/your_dataset'

# The following 4 lines are the only parts of the code that you need to change. You can simply run the rest.
data_dir      = os.path.join(GDRIVE_HOME,FOLDER) ## path of the general experiment
initial_model = os.path.join(GDRIVE_HOME,'MLVU/res18_vggface1_baseline.model') ## path to the pre-trained model
test_zip      = os.path.join(data_dir,'test_data.zip') ## test data as zip

# Extract the cropped images
with ZipFile(test_zip, 'r') as zipObj:
  zipObj.extractall("/test_set")
print('Zip extraction complete')

Make sure that the files have been extracted properly. Make sure that this is a reasonable number.

In [ ]:
test_files  = glob.glob('/test_set/*/*.jpg')
print(len(test_files),'test set files found.')

First, clone the face recognition trainer from GitHub and add it to path.

In [ ]:
!rm -rf face_trainer
!git clone https://github.com/joonson/face_trainer.git

sys.path.append('face_trainer')

The evaluation script. Please do not change, but try to read and understand.

In [ ]:
import datetime
from utils import *
from EmbedNet import *
import torchvision.transforms as transforms

# ## ===== ===== ===== ===== ===== ===== ===== =====
# ## Evaluation script
# ## ===== ===== ===== ===== ===== ===== ===== =====

def test_network(args):

    ## Load models
    s = EmbedNet(**vars(args)).cuda();

    ## Input transformations for evaluation
    test_transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Resize(256),
         transforms.CenterCrop([224,224]),
         transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

    ## Initialise trainer
    trainer     = ModelTrainer(s, **vars(args))

    ## Load model weights
    trainer.loadParameters(args.initial_model);
    print("Model %s loaded!"%args.initial_model);

    sc, lab = trainer.evaluateFromList(transform=test_transform, **vars(args))
    result = tuneThresholdfromScore(sc, lab, [1, 0.1]);

    print('EER %2.4f'%(result[1]))

Use the script below to evaluate the trained network.


In [ ]:
import easydict 
args = easydict.EasyDict({"nDataLoaderThread": 5, 
                          "initial_model": initial_model,
                          "model": "ResNet18",
                          "nOut": 512,
                          "test_path": "/test_set",
                          "test_list": "/test_set/test_list.csv",
                          # The arguments below are necessary to initiate the trainer instance, 
                          # but do not affect the test phase.
                          "trainfunc": "softmax",
                          "optimizer": "adam",
                          "scheduler": "steplr",
                          "lr": 0.001,
                          "lr_decay": 1,
                          "test_interval": 5,
                          "max_epoch": 50,
                          "weight_decay": 0,
                          "nPerClass": 1,
                          "nClasses": 2622,
                          "mixedprec": False})
        
test_network(args)